# Text Mining 

## 1. Préprocessing

In [ ]:
import pandas as pd

df = pd.read_parquet("data_clustered.parquet")
print(f"Données chargées : {len(df)} photos")
print(f"Nombre de clusters : {df['cluster_hdbscan'].nunique() - 1}")

### Définition des stopwords

On définit manuellement une liste de stopwords, contenant les mots non-pertinents pour identifier les lieux touristiques de Lyon.

In [ ]:
stopwords_from_method1 = {
    "lyon", "france", "img", "jpg", "uploaded", "europe", "square", "iphone", "instagram",
    "franca", "rhonealpes", "geotagged", "janvier", "fevrier", "mars", "avril", "mai", "juin",
    "juillet", "aout", "septembre", "octobre", "novembre", "decembre", "interieur", "live",
    "des", "squareformat", "dsc", "iphoneography", "art", "instagramapp", "foursquare",
    "venue", "japan", "flickrmobile", "french", "francia", "dscf", "frankrijk", "frankreich",
    "flickriosapp", "touch"
}

stopwords_from_method2 = {
    "ngc", "paysage", "landscape", "upload", "ios", "flickr", "mobile", "app", "filter",
    "blackandwhite", "black", "white", "phone", "noir", "blanc", "the", "nos",
    "canon", "mmf", "monochrome", "alpes", "rhones", "photo", "urban", "photos", "nikon",
    "night", "auvergne", "urbain", "photography", "town", "city", "les", "street",
    "architecture", "ville", "reich", "frankreich", "sky", "metropolisoflyon",
    "burgundy", "bourgogne", "auvergnerhonealpes", "geo", "and", "lat", "lon", "lione",
    "eme", "rhone", "nuit", "noiretblanc", "light", "people"
}

# Unifier les 2 ensembles
stopwords = stopwords_from_method1.union(stopwords_from_method2)

print(f"Nombre total de stopwords : {len(stopwords)}")

### Tokenization : Mis de côté, résultats peu probants

La tokenization est mise de côté car les résultats obtenus ne sont pas pertinents : d'après nos tests, nous n'avons pas trouvé de librairie permettant de segmenter correctement les "hashtags" en mots significatifs. Par exemple, "basiliquedefourviere" n'est pas segmenté en "basilique de fourviere", ou bien l'introduction de segmentation dégrade le reste des tags déjà correctement segmentés.

La partie "segmentation" est donc à ignorer.

On applique une tokenization afin de mieux traiter les hashtags, par exemple : "fetedeslumieres" devient "fete", "des", et "lumieres".

In [ ]:
# import sys
# import subprocess
# import unicodedata

# # 1. Installation de ekphrasis ET pyspellchecker
# try:
#     from ekphrasis.classes.segmenter import Segmenter
#     from spellchecker import SpellChecker
# except ImportError:
#     print("Installation des librairies nécessaires...")
#     subprocess.check_call([sys.executable, "-m", "pip", "install", "ekphrasis", "pyspellchecker"])
#     from ekphrasis.classes.segmenter import Segmenter
#     from spellchecker import SpellChecker

# # 2. Chargement 
# try:
#     # On utilise le corpus 'twitter' qui marche pour les hashtags
#     seg_tw = Segmenter(corpus="twitter")
#     spell_fr = SpellChecker(language='fr') 
#     print("✓ Segmenter Ekphrasis chargé")
#     print("✓ Dictionnaire Français chargé")
# except Exception as e:
#     print(f"Erreur au chargement: {e}")

### Fonction de nettoyage du texte

On crée une fonction qui :
1. Concatène title et tags
2. Met en minuscules
3. Supprime les accents (é → e, ç → c)
4. Supprime la ponctuation et caractères spéciaux
5. Filtre les stopwords et mots trop courts

In [ ]:
import re
import unicodedata

def clean_text(title, tags):
    # Combiner title et tags
    text = ""
    if isinstance(title, str):
        text += title + " "
    if isinstance(tags, str):
        text += tags
    
    if not text.strip():
        return []
    
    # 1. Minuscules
    text = text.lower()
    
    # 2. Supprimer accents
    text = unicodedata.normalize('NFD', text)
    text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
    
    # 3. Ne garder que lettres et espaces
    text = re.sub(r'[^a-z\s]', ' ', text)
    
    # 4. Split initial
    words = text.split()
    
    # 5. Segmentation Intelligente avec Ekphrasis
    # LA SEGMENTATION EST MISE DE CÖTÉ (voir note plus haut)
    # expanded_words = []
    # for w in words:
    #     # STRATÉGIE :
    #     # 1. Si c'est un mot français valide -> On garde tel quel
    #     # 2. Sinon -> On demande à Ekphrasis de le segmenter
        
    #     if (w in spell_fr) or (len(w) < 4):
    #         expanded_words.append(w)
    #     else:
    #         try:
    #             # ekphrasis renvoie une string avec des espaces "mot1 mot2"
    #             segmented = seg_tw.segment(w)
    #             expanded_words.extend(segmented.split())
    #         except:
    #             expanded_words.append(w)
    
    # 6. Filtrer stopwords et mots courts (re-vérification après segmentation)
    expanded_words = words  # Sans segmentation
    final_words = [w for w in expanded_words if w not in stopwords and len(w) > 2]
    
    # 7. Uniques
    unique_words = list(dict.fromkeys(final_words))
    
    return unique_words

# Test de la fonction de nettoyage
print(clean_text("Visite de la Basilique de Fourvière en été !", "basiliquedefourviere lyon france summer travel"))

### Application du nettoyage

On applique la fonction de nettoyage sur toutes les photos pour créer une colonne unique `cleaned_text`.

**Note** : Cette étape peut prendre quelques minutes avec `wordsegment`. Une barre de progression s'affichera pour suivre l'avancement.

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

output_file = "data_clustered_tokenized.parquet"

# LA SEGMENTATION EST MISE DE CÖTÉ (voir note plus haut)
# # On évite de refaire la tokenization (qui prend du temps) si c'est déjà fait
# if os.path.exists(output_file):
#     print(f"🔄 Chargement des données tokenisées depuis '{output_file}'...")
#     df = pd.read_parquet(output_file)
#     print("✅ Données chargées avec succès ! (Traitement passé)")
# else:
#     print(f"🚀 Démarrage du traitement (Nettoyage + Tokenisation)...")
#     tqdm.pandas(desc="Tokenisation en cours")
    
#     # Appliquer le nettoyage sur title + tags avec barre de progression
#     df['cleaned_text'] = df.progress_apply(
#         lambda row: clean_text(row['title'], row['tags']), 
#         axis=1
#     )
    
#     print(f"💾 Sauvegarde du résultat dans '{output_file}'...")
#     df.to_parquet(output_file)
#     print("✅ Traitement terminé et sauvegardé !")

df['cleaned_text'] = df.progress_apply(
    lambda row: clean_text(row['title'], row['tags']), 
    axis=1
)

# Statistiques sur le nettoyage
total_photos = len(df)
photos_avec_mots = (df['cleaned_text'].str.len() > 0).sum()
photos_sans_mots = total_photos - photos_avec_mots

print(f"\n📊 Statistiques :")
print(f"  - Photos avec mots : {photos_avec_mots} ({photos_avec_mots/total_photos*100:.1f}%)")
print(f"  - Photos sans mots : {photos_sans_mots} ({photos_sans_mots/total_photos*100:.1f}%)")
print(f"  - Nombre moyen de mots/photo : {df['cleaned_text'].str.len().mean():.1f}")

### Vérification du résultat

On affiche quelques exemples pour vérifier que le nettoyage fonctionne correctement.

In [ ]:
# Afficher quelques exemples
sample = df[df['cleaned_text'].str.len() > 0].sample(n=10, random_state=42)

print("Exemples de textes nettoyés :\n")
for idx, row in sample.iterrows():
    print(f"Title original : {row['title'][:80]}...")
    print(f"Tags originaux : {row['tags'][:80] if isinstance(row['tags'], str) else 'N/A'}...")
    print(f"Mots nettoyés  : {row['cleaned_text'][:10]}")  # 10 premiers mots
    print(f"Cluster        : {row['cluster_hdbscan']}")
    print("-" * 80)

## 2. Méthode 1 : Mot le plus fréquent par cluster

### Principe

Pour chaque cluster, on identifie le mot le plus fréquent dans les textes nettoyés (titles + tags).

**Hypothèse** : Le mot dominant représente le lieu ou le monument principal du cluster.

### Fonction de calcul des mots dominants

In [ ]:
from collections import Counter

def find_cluster_keywords(df, cluster_col='cluster_hdbscan', text_col='cleaned_text', top_k=1):
    """
    Trouve le(s) mot(s) le(s) plus fréquent(s) pour chaque cluster.
    
    Args:
        df: DataFrame avec colonnes cluster et texte nettoyé
        cluster_col: Nom de la colonne contenant les labels de cluster
        text_col: Nom de la colonne contenant les listes de mots nettoyés
        top_k: Nombre de mots à retourner par cluster
    
    Returns:
        dict {cluster_id: mot_dominant}
    """
    cluster_keywords = {}
    
    for cluster_id in sorted(df[cluster_col].unique()):
        if cluster_id == -1:
            continue  # Ignorer le bruit
        
        # Récupérer tous les mots du cluster
        cluster_data = df[df[cluster_col] == cluster_id]
        all_words = []
        
        for words_list in cluster_data[text_col].dropna():
            all_words.extend(words_list)
        
        # Si pas de mots, mettre "unknown"
        if not all_words:
            cluster_keywords[cluster_id] = "unknown"
            continue
        
        # Compter les occurrences
        word_counts = Counter(all_words)
        
        # Prendre le mot le plus fréquent
        top_word = word_counts.most_common(top_k)[0][0]
        cluster_keywords[cluster_id] = top_word
    
    return cluster_keywords

# Calculer les mots-clés dominants
cluster_keywords = find_cluster_keywords(df)

# Ajouter au DataFrame
df['cluster_keyword'] = df['cluster_hdbscan'].map(cluster_keywords).fillna("noise")

print(f"✓ Mots-clés calculés pour {len(cluster_keywords)} clusters")

### Statistiques et aperçu des clusters

In [ ]:
# Créer le résumé des clusters
cluster_summary = df[df['cluster_hdbscan'] != -1].groupby('cluster_hdbscan').agg({
    'cluster_keyword': 'first',
    'id': 'count'
}).rename(columns={'id': 'nb_photos'}).sort_values('nb_photos', ascending=False)

n_clusters = len(cluster_summary)

# Compter les mots-clés par CLUSTER (pas par photo)
keyword_cluster_counts = cluster_summary['cluster_keyword'].value_counts()
n_unique_keywords = (keyword_cluster_counts == 1).sum()
n_duplicate_keywords = len(keyword_cluster_counts[keyword_cluster_counts > 1])

print(f"🔍 Qualité des mots-clés:")
print(f"  - Mots-clés uniques: {n_unique_keywords}/{n_clusters} ({n_unique_keywords/n_clusters*100:.1f}%)")
print(f"  - Mots-clés dupliqués: {n_duplicate_keywords} (partagés par plusieurs clusters)")
print(f"  ⚠️  Ambiguïté: {n_clusters - n_unique_keywords} clusters partagent un même mot-clé")

# Top des mots-clés les plus fréquents (candidats stopwords)
print(f"\n🚫 Mots-clés les plus fréquents (candidats pour stopwords):")
print(f"    Ces mots-clés dominent plusieurs clusters, ils sont donc peu distinctifs\n")
for keyword, count in keyword_cluster_counts.head(20).items():
    print(f"    '{keyword}' : {count} clusters")

print(f"\n📍 Top 20 clusters par taille :\n")
print(cluster_summary.head(20).to_string())

### Visualisation sur carte interactive

On affiche les clusters sur une carte avec leurs mots-clés dominants pour identifier visuellement les zones d'intérêt.

In [ ]:
import folium

# Échantillonner pour la performance
sample_size = 30000
sample = df.sample(n=min(sample_size, len(df)), random_state=0)

# Créer la carte
m = folium.Map(
    location=[df["lat"].median(), df["long"].median()],
    zoom_start=12,
    tiles="CartoDB positron"
)

# Palette de couleurs
palette = [
    "red", "blue", "green", "purple", "orange",
    "darkred", "lightred", "beige", "darkblue",
    "darkgreen", "cadetblue", "darkpurple",
    "pink", "lightblue", "lightgreen",
    "gray", "black", "lightgray"
]

# Ajouter les points
for _, r in sample.iterrows():
    cluster = r["cluster_hdbscan"]
    
    if cluster == -1:
        continue
    else:
        color = palette[cluster % len(palette)]
        keyword = r["cluster_keyword"]
    
    folium.CircleMarker(
        location=[r["lat"], r["long"]],
        radius=2,
        color=color,
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(
            f"""<b>Keyword:</b> {keyword}<br/>
               <b>Cluster:</b> {cluster}<br/>
               <a href="{r.get('url', '#')}" target="_blank">Open Flickr</a>""",
            max_width=250
        )
    ).add_to(m)

# Titre sur la carte
title_html = '''
<div style="position: fixed; top: 10px; left: 50px; width: 400px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
     <b>Méthode 1 : Mot le plus fréquent par cluster</b>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Sauvegarder
m.save("output/text_mining_method1.html")

### Analyse des résultats

**Observations** :

En explorant la carte, on peut identifier les mots-clés dominants pour chaque zone touristique. Certains mots sont pertinents (ex: "croixrousse", "bellecour", "jacobins"), d'autres sont trop génériques ou non-informatifs.

**Limites de cette méthode** :

1. **Sensibilité aux stopwords** : La qualité dépend fortement de la liste de stopwords. Des mots non-pertinents peuvent dominer si on ne les filtre pas manuellement, ce qui est fastidieux et propice aux erreurs.

2. **Mot unique par cluster** : Un seul mot ne suffit pas toujours à caractériser un lieu (ex: "Part Dieu" → seulement "part" ou "dieu").

3. **Approche itérative** : Nécessite d'inspecter visuellement les résultats et d'ajuster manuellement les stopwords pour améliorer la qualité.

4. **Pas de pondération contextuelle** : Tous les mots ont le même poids, qu'ils soient spécifiques (ex: "fourviere") ou génériques (ex: "rue", "statue").

**Amélioration possible** : Utiliser TF-IDF pour pondérer l'importance des mots en fonction de leur spécificité à chaque cluster, plutôt que simplement compter les fréquences brutes.

## 3. Méthode 2 : TF-IDF

Pour pallier les limites de la simple fréquence, on utilise **TF-IDF** (Term Frequency - Inverse Document Frequency).

**Principe** : TF-IDF donne un score élevé aux mots :
- Fréquents dans un cluster spécifique (TF)
- Rares dans les autres clusters (IDF)

Cela permet de capturer les mots **distinctifs** plutôt que simplement fréquents.

### Préparation des textes

On transforme chaque photo en un texte, puis on concatène toutes les photos d’un cluster en un seul document.

In [ ]:
# 1) Texte par photo
df['combined_text'] = df['cleaned_text'].apply(lambda words: ' '.join(words))

# 2) Document par cluster
cluster_documents = df[df['cluster_hdbscan'] != -1].groupby('cluster_hdbscan')['combined_text'] \
    .apply(lambda texts: ' '.join(texts)) \
    .reset_index()

cluster_documents.columns = ['cluster', 'document']
cluster_documents.head()

On regarde le nombre de clusters et la taille moyenne des documents.

In [ ]:
print(f"Nombre de clusters : {len(cluster_documents)}")
print(f"Taille moyenne des documents : {cluster_documents['document'].str.len().mean():.0f} caractères")

### Construction de la matrice TF‑IDF

On garde uniquement les mots utiles :
- `min_df=2` : le mot doit apparaître dans au moins 2 clusters
- `max_df=0.8` : on supprime les mots trop communs

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    min_df=2,
    max_df=0.8
)

tfidf_matrix = tfidf.fit_transform(cluster_documents['document'])
feature_names = tfidf.get_feature_names_out()

print(f"Matrice TF‑IDF : {tfidf_matrix.shape}")

### Extraction des top mots par cluster

On crée une fonction réutilisable pour extraire les mots les plus représentatifs.

In [ ]:
import numpy as np

def get_top_terms_for_cluster(cluster_idx, top_n=3):
    scores = tfidf_matrix[cluster_idx].toarray().flatten()
    top_indices = np.argsort(scores)[-top_n:][::-1]
    return feature_names[top_indices]

# Associer les top mots à chaque cluster
tfidf_labels = {}

for i in range(len(cluster_documents)):
    cluster_id = cluster_documents.iloc[i]['cluster']
    top_terms = get_top_terms_for_cluster(i, top_n=4)
    tfidf_labels[cluster_id] = " / ".join(top_terms)

# Ajouter au DataFrame
df['cluster_tfidf'] = df['cluster_hdbscan'].map(tfidf_labels).fillna("noise")

print("✓ Mots-clés TF‑IDF ajoutés au dataframe")

### Visualisation des clusters (résumé final)

On visualise les clusters sur une carte avec leurs labels TF‑IDF.

In [ ]:
import folium

# Échantillonner pour la performance (comme avant)
sample_size = 30000
sample = df.sample(n=min(sample_size, len(df)), random_state=0)

m = folium.Map(
    location=[df["lat"].median(), df["long"].median()],
    zoom_start=12,
    tiles="CartoDB positron"
)

palette = [
    "red", "blue", "green", "purple", "orange",
    "darkred", "lightred", "beige", "darkblue",
    "darkgreen", "cadetblue", "darkpurple",
    "pink", "lightblue", "lightgreen",
    "gray", "black", "lightgray"
]

for _, r in sample.iterrows():
    cluster = r["cluster_hdbscan"]
    if cluster == -1:
        continue
    
    color = palette[cluster % len(palette)]
    label = r["cluster_tfidf"]
    
    folium.CircleMarker(
        location=[r["lat"], r["long"]],
        radius=2,
        color=color,
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(
            f"""<b>TF‑IDF:</b> {label}<br/>
               <b>Cluster:</b> {cluster}<br/>
               <a href="{r.get('url', '#')}" target="_blank">Open Flickr</a>""",
            max_width=250
        )
    ).add_to(m)

title_html = '''
<div style="position: fixed; top: 10px; left: 50px; width: 400px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
     <b>Méthode 2 : TF‑IDF (top mots par cluster)</b>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

m.save("output/text_mining_tfidf.html")

### Analyse des résultats

**Points forts** :
- Les mots sont souvent plus spécifiques que la méthode 1.
- Moins dépendant des stopwords manuels.

**Limites** :
- La qualité dépend de la préparation (cleaning + stopwords).

TF‑IDF donne une base solide pour nommer les clusters, mais il faut toujours ajuster avec une inspection manuelle pour ajuster les stopwords.

## 4. Méthode 3 : Règles d'Association

### Principe

Les **règles d'association** découvrent des **combinaisons de mots** qui apparaissent fréquemment ensemble dans les clusters.

**Différence avec les méthodes précédentes :**
- Méthode 1 : un seul mot dominant → ex: "basilique"
- Méthode 2 (TF-IDF) : liste de mots distinctifs → ex: "basilique, fourviere, colline"
- **Méthode 3** : associations entre mots → ex: **"basilique + fourviere"** (vont souvent ensemble)

On peut ainsi capturer les noms composés et expressions ("place bellecour", "parc tete d'or") plutôt que des mots isolés.

On commence avec l'algorithme Apriori, qui génère des règles du type `{mot1, mot2} → {mot3}` avec des scores de confiance.

### Préparation des transactions

On transforme chaque cluster en une "transaction" (liste de mots uniques).

In [ ]:
# Créer les transactions : 1 transaction = tous les mots d'un cluster
transactions = []

for cluster_id in sorted(df['cluster_hdbscan'].unique()):
    if cluster_id == -1:
        continue
    
    # Récupérer tous les mots du cluster
    cluster_data = df[df['cluster_hdbscan'] == cluster_id]
    all_words = []
    
    for words_list in cluster_data['cleaned_text'].dropna():
        all_words.extend(words_list)
    
    # Garder les mots uniques
    unique_words = list(set(all_words))
    transactions.append(unique_words)

print(f"✓ {len(transactions)} transactions créées")
print(f"  Exemple (cluster 0) : {transactions[0][:10]}...")

L'algorithme Apriori nécessite un format binaire : chaque ligne = 1 transaction, chaque colonne = 1 mot.

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

print(f"Matrice encodée : {df_encoded.shape}")
df_encoded.head()

### Apriori avec plusieurs paramètres

Le **support** minimal définit la fréquence minimale d'apparition d'une combinaison de mots.

On teste plusieurs valeurs pour trouver le bon équilibre :
- Trop bas → trop de règles (bruit)
- Trop haut → pas assez de règles (perte d'info)

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np

support_values = [0.125, 0.15, 0.25, 0.5, 0.75]
results = []

for min_sup in support_values:
    print(f"Début pour min_support = {min_sup}")
    
    frequent_itemsets = apriori(
        df_encoded,
        min_support=min_sup,
        use_colnames=True,
        max_len=3
    )
    
    # Ajouter length
    if len(frequent_itemsets) > 0:
        frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(len)
        max_len = frequent_itemsets['length'].max()
    else:
        max_len = 0
    
    # Générer des règles si possible
    if len(frequent_itemsets) > 0:
        rules = association_rules(
            frequent_itemsets,
            metric="confidence",
            min_threshold=0.6,
            num_itemsets=len(transactions)
        )
        mean_lift = rules['lift'].mean() if len(rules) > 0 else 0
        mean_conf = rules['confidence'].mean() if len(rules) > 0 else 0
    else:
        rules = pd.DataFrame()
        mean_lift = 0
        mean_conf = 0
    
    # Couverture : % de clusters ayant au moins un itemset de taille >=2
    if len(frequent_itemsets) > 0:
        itemsets_2plus = frequent_itemsets[frequent_itemsets['length'] >= 2]['itemsets']
        covered = 0
        for cluster_words in transactions:
            if any(itemset.issubset(set(cluster_words)) for itemset in itemsets_2plus):
                covered += 1
        coverage = covered / len(transactions)
    else:
        coverage = 0
    
    results.append({
        'min_support': min_sup,
        'n_itemsets': len(frequent_itemsets),
        'max_length': max_len,
        'mean_lift': mean_lift,
        'mean_confidence': mean_conf,
        'coverage': coverage
    })
    
    print(f"  → itemsets: {len(frequent_itemsets)} | rules: {len(rules)} | coverage: {coverage:.2f}")

results_df = pd.DataFrame(results)
results_df

On visualise le résultat de l'exploration avec plusieurs valeurs de paramètres :

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 2, figsize=(15, 8))

# 1) Quantité d’itemsets
ax[0][0].plot(results_df['min_support'], results_df['n_itemsets'], marker='o', linewidth=2)
ax[0][0].set_title("Quantité d'itemsets")
ax[0][0].set_xlabel("min_support")
ax[0][0].set_ylabel("Nombre d'itemsets")
ax[0][0].grid(True, alpha=0.3)

# 2) Lift
ax[0][1].plot(results_df['min_support'], results_df['mean_lift'], marker='s', label='Lift moyen')
ax[0][1].set_title("Qualité des règles : Lift")
ax[0][1].set_xlabel("min_support")
ax[0][1].set_ylabel("Valeur moyenne")
ax[0][1].legend()
ax[0][1].grid(True, alpha=0.3)

# 3) Confidence
ax[1][0].plot(results_df['min_support'], results_df['mean_confidence'], marker='^', label='Confidence moyenne')
ax[1][0].set_title("Qualité des règles : Confidence")
ax[1][0].set_xlabel("min_support")
ax[1][0].set_ylabel("Valeur moyenne")
ax[1][0].legend()
ax[1][0].grid(True, alpha=0.3)

# 4) Couverture
ax[1][1].plot(results_df['min_support'], results_df['coverage'], marker='o', color='green')
ax[1][1].set_title("Couverture des clusters")
ax[1][1].set_xlabel("min_support")
ax[1][1].set_ylabel("Proportion couverte")
ax[1][1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Analyse des graphiques et choix des paramètres

L'analyse des courbes ci-dessus met en évidence trois compromis majeurs pour le choix du `min_support` :

1.  **Explosion combinatoire (Quantité)** : Le nombre d'itemsets diminue de manière exponentielle. Avec un support de **0.10**, on obtient près de 300 000 itemsets, ce qui est très coûteux en calcul et contient probablement beaucoup de "bruit". À **0.20**, ce nombre chute à environ 20 000, rendant l'analyse beaucoup plus rapide.

2.  **Qualité des règles (Lift & Confidence)** : On observe que la qualité moyenne des règles diminue lorsque le support augmente. 
    *   Les associations très fréquentes (support élevé) sont souvent génériques, d'où un *Lift* plus faible (~1.50).
    *   Les associations plus rares (support faible) sont souvent plus fortes et spécifiques (ex: "basilique" + "fourviere"), avec un *Lift* plus élevé (~1.90).

3.  **Couverture des clusters** : La couverture reste excellente (> 96%) jusqu'à un support de **0.20**. Elle ne commence à chuter significativement qu'après 0.20.

**Conclusion** : 
Le point d'équilibre ("le coude") semble se situer autour de **0.20**.
*   On élimine l'explosion combinatoire (division par 6 du nombre d'itemsets).
*   On maintient une excellente couverture (~96.5% des clusters sont labellisés).
*   On conserve une qualité de règles acceptable.

Nous allons donc retenir **`min_support = 0.20`** pour la génération finale des règles.

### Génération des règles d'association

On génère les règles avec une confiance minimale de 60%.

In [ ]:
from mlxtend.frequent_patterns import association_rules

# Générer les itemsets fréquents avec min_support = 0.5
frequent_itemsets = apriori(
    df_encoded,
    min_support=0.5,
    use_colnames=True,
    max_len=10
)

# Puis générer les règles
rules = association_rules(
    frequent_itemsets, 
    metric="confidence", 
    min_threshold=0.5, 
    num_itemsets=len(transactions)
)
rules = rules.sort_values('lift', ascending=False)

print(f"✓ {len(rules)} règles générées")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(100).to_string())

### Top règles par lift

Le **lift** mesure la force de l'association (lift > 1 = association positive).

In [ ]:
# Top 10 règles
top_rules = rules.head(100).copy()

# Formater pour l'affichage
top_rules['rule'] = top_rules.apply(
    lambda row: f"{set(row['antecedents'])} → {set(row['consequents'])}", 
    axis=1
)

print(top_rules[['rule', 'support', 'confidence', 'lift']].to_string())

Au vu de la faible qualité des règles d'association trouvées, nous n'utiliserons pas cette méthode pour labelliser les clusters.

# 5. LLMs

Au vu des mauvais résultats donnés par les règles d'association, on tente l'utilisation d'un LLM pour labelliser les clusters, à partir des 5 mots les plus significatifs extraits par TF-IDF.

In [ ]:
import requests
import json
import time
from tqdm import tqdm

def generate_title_with_ollama(keywords_list, model="mistral"):
    keywords_str = ", ".join(keywords_list)
    
    prompt = f"""
    Tu es un assistant utile pour nommer des clusters touristiques.
    À partir de ces mots-clés décrivant un lieu à Lyon (France) : "{keywords_str}".
    Génère un titre très court et naturel (max 5 mots) en français.
    Le titre doit refléter le nom du lieu, sans s'attacher aux mots-clés génériques
    qui ne sont pas spécifiques à ce lieu.
    **Retourne UNIQUEMENT le titre, sans guillemets ni explications.**
    """

    url = "http://localhost:11434/api/generate"
    data = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    try:
        response = requests.post(url, json=data)
        if response.status_code == 200:
            return response.json()['response'].strip()
        else:
            return "Error LLM"
    except Exception as e:
        return f"Connection Failed: {e}"

In [ ]:
from IPython.display import clear_output

# Dictionary to store LLM generated titles
llm_titles = {}

# Get unique cluster IDs (excluding noise -1)
unique_clusters = sorted(df['cluster_hdbscan'].unique())
if -1 in unique_clusters: unique_clusters.remove(-1)

print(f"Generating titles for {len(unique_clusters)} clusters using Ollama...")

for cluster_id in tqdm(unique_clusters):
    # Retrieve the top terms using your existing function index logic
    # Note: Ensure the index matches logical position in cluster_documents
    try:
        # Find the index in cluster_documents corresponding to this cluster_id
        doc_idx = cluster_documents.index[cluster_documents['cluster'] == cluster_id][0]
        keywords = get_top_terms_for_cluster(doc_idx, top_n=4)
        
        # Generate title
        human_title = generate_title_with_ollama(keywords)
        
        clear_output(wait=True)
        print(f"Cluster {cluster_id}: Keywords: {keywords} -> Title: {human_title}")
        
        llm_titles[cluster_id] = human_title
    except IndexError:
        llm_titles[cluster_id] = "unknown"

# Add to DataFrame
df['cluster_llm_title'] = df['cluster_hdbscan'].map(llm_titles).fillna("noise")

print("✓ LLM Titles generated!")
print(df[['cluster_hdbscan', 'cluster_tfidf', 'cluster_llm_title']].drop_duplicates().head(10))

In [ ]:
import folium

# Échantillonner pour la performance
sample_size = 30000
sample = df.sample(n=min(sample_size, len(df)), random_state=0)

m_llm = folium.Map(
    location=[df["lat"].median(), df["long"].median()],
    zoom_start=12,
    tiles="CartoDB positron"
)

# Palette de couleurs
palette = [
    "red", "blue", "green", "purple", "orange",
    "darkred", "lightred", "beige", "darkblue",
    "darkgreen", "cadetblue", "darkpurple",
    "pink", "lightblue", "lightgreen",
    "gray", "black", "lightgray"
]

for _, r in sample.iterrows():
    cluster = r["cluster_hdbscan"]
    if cluster == -1:
        continue
    
    color = palette[cluster % len(palette)]
    
    # On récupère le titre LLM et les mots-clés TF-IDF
    llm_title = r.get("cluster_llm_title", "Unknown")
    tfidf_kw = r.get("cluster_tfidf", "N/A")
    
    folium.CircleMarker(
        location=[r["lat"], r["long"]],
        radius=2,
        color=color,
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(
            f"""<b>Title:</b> {llm_title}<br/>
               <span style='font-size:0.8em; color:gray'>Keywords: {tfidf_kw}</span><br/>
               <b>Cluster:</b> {cluster}<br/>
               <a href="{r.get('url', '#')}" target="_blank">Open Flickr</a>""",
            max_width=300
        )
    ).add_to(m_llm)

title_html = '''
<div style="position: fixed; top: 10px; left: 50px; width: 450px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
     <b>Méthode 3 : Titres générés par IA (Ollama)</b><br>
     <i style="font-size:12px">Basé sur les mots-clés TF-IDF</i>
</div>
'''
m_llm.get_root().html.add_child(folium.Element(title_html))

m_llm.save("output/text_mining_llm.html")
print("Carte sauvegardée dans output/text_mining_llm.html")